In [319]:
# import all neccessary libs
import pandas as pd
import numpy as np
import glob
from datetime import datetime

# giving cols name to each data and features
name=['Date','Latitude', 'Longitude', 'Altitude', 'RPM',
        'Driver Demand Torque (%)', 'Engine Load (%)', 'Engine Torque Mode', 'TPS (%)',
        'Percent Load Curret Speed', 'Fuel Rate (L-Hr)', 'Vehicle Speed',
        'Inj Q Cur (mg-st)', 'Inj Q Tor (mg-st)', 'Boost Pressure (mBar)',
        'Atmospheric Pressure (mBar)', 'Coolant Temperature (*C)',
        'Oil Temperature (*C)', 'Boost Temperature (*C)', 'Oil Pressure (mBar)',
        'Battery Voltage (V)', 'Cam Speed (rpm)', 'Rail Pressure (mBar)',
        'Rail Pressure set (mBar)', 'MU PWM (%)', 'MU Vol (mm3-s)',
        'Torque Rat', 'Torque (Nm)', 'TQ Limit Set', 'Main Injection (mg-st)',
        'Pilot Injection (mg-st)', 'Pos 2 Injector (mg-st)', 'EGR Prop (%)',
        'EGR Pos D (%)', 'EGR Pos A (%)', 'Clutch Switch', 'Brake Switch',
        'Engine Grad (rpm)', 'param1', 'param2','col_no'
]


# Data Reader from folder
def data_reader():
    # iterate over all the '.csv' files kept in the directory
    for file_name in glob.glob(directoryPath+'*.csv'):
        # read the csv file
        data_csv = pd.read_csv(file_name,sep = ',',names=name, index_col=0)
        # reset the index
        data_csv.reset_index(inplace=True)
        # store them into the read_data_stg list
        read_data_stg.append(data_csv)
        print(f"Reading Complete --->> {file_name}" )
        print("*"*10)
        print()
    return read_data_stg

# date finder from csv file
def date_time_sep(df):
    # to find the start and end date
    # search term
    search_term = ['START', 'END']
    # value stg
    ans_ = []
    # iterate over all the search term to find the value
    for ser in search_term:
        # checking the rows that contain the search term 
        check = df[df['Date'].str.contains(ser)]
        # resetingtthe index of found row
        check.reset_index(inplace=True)
        # finding the value of date from (EX:- 'START DATE:040521') after colon
        val_of_date = check['Date'][0].split(': ')[1]
        # making them into appropriate form  --> 04-05-21
        date_conv = val_of_date[0:2]+'-' + val_of_date[2:4] + '-' + val_of_date[4:]
        final_date = datetime.strptime(date_conv, '%d-%m-%y').date()
        # finally converting into year
        val = final_date.strftime("%d-%m-%Y")
        # saving the data
        ans_.append(val)
    return ans_[0]

# RPM value convertor
def rpm_cal(n, muliplier, offset):
    # default values of multiplier
    mul = muliplier
    # default values of offset
    offs = offset
    # slicing the LSB and MSB bit and joing them again EX:- 4028 to 2840
    val1 = str(n[2:])+str(n[:2])
    try:
        # finding the hexadecimal to decimal value
        # if value in string 
        final_val = int(val1, 16)
    except:
        # finding the hexadecimal to decimal value
        # if value not in string convert it into int 
        # and then string
        final_val = int(str(int(val1)), 16)
    # finally applies the formula for final conversion    
    val = (final_val * mul) + offs
    return val

# Latitude value convertor
def lat_convs(num):
    # convert it into flot for base checking of 'NaN'
    num_chek = float(num)
    # cheking of 'NaN' value
    if np.isnan(num_chek)==False:
        # conversion to string
        num = str(num)
        # converting from ddmm.mmmmm to dd.dddddd
        val_lat = str(num[:2]) + str(float(num[2:])/60)[1:]
        return float(val_lat)
    else:
        # if data is 'NaN' return as_it_is
        return num
    
# longitude conversion
def long_convs(num):
    # convert it into flot for base checking of 'NaN'
    num_chek = float(num)
    # cheking of 'NaN' value
    if np.isnan(num_chek)==False:
        # conversion to string
        num = str(num)
        # converting from ddmm.mmmmm to dd.dddddd
        val_long = str(num[:3]) + str(float(num[3:])/60)[1:]
        return float(val_long)
    else:
        # if data is 'NaN' return as_it_is
        return num

# conversion for fuel rate columns
def convSingleCol(n, muliplier, offset):
    
    # default values of multiplier
    mul = muliplier
    # default values of offset
    offs = offset
    
    # taking LSB and MSB bits
    val1 = n[:2]
    val2 = n[2:]
    
    try:
        # converting the MSB and LSB bits seprately
        hex_dec1 = int(val1, 16)
        hex_dec2 = int(val2, 16)
        # then joing them 
        final_val = hex_dec1 + hex_dec2
    except:
        # converting the MSB and LSB bits seprately
        hex_dec1 = int(str(int(val1)), 16)
        hex_dec2 = int(str(int(val2)), 16)
        # then joing them 
        final_val = hex_dec1 + hex_dec2
    # finally applies the formula for final conversion    
    val = (final_val * mul) + offs
    return val


# base function for conversion
def ConvHexToDec(n, muliplier, offset):
    # default values of multiplier
    mul = muliplier
    # default values of offset
    offs = offset
    
    try:
        # hex to dec
        hex_dec = int(n, 16)
    except:
        # hex to dec
        hex_dec = int(str(int(n)), 16)
    # final value    
    val = (hex_dec * mul) + offs
    return val

# Pre-processing of the data to remove cols and null values
def pre_processing_data_and_date_finder(data_file):
    # data storage
    pre_processed_data_stg = []
    date_stg = []
    flag=0
    
    # loop over all the df one by one
    for dfitem in data_file:
        print("Pre-processing file --> ",flag)
        # calculate the start and end date
        val = date_time_sep(dfitem)
        # store it
        date_stg.append(val)
        # drop these unwanted cols
        dfitem.drop(['Date', 'col_no'], axis=1,inplace=True)
        # remove those rows which contain more than 5 null values
        # we can change as per our need
        # 3 null possible due to lat,lon, and altitude val
        dfitem = dfitem[dfitem.isnull().sum(axis=1) < 5]
        # storing the value of dataframe
        pre_processed_data_stg.append(dfitem)
        flag = flag+1
    return pre_processed_data_stg,date_stg


# base function to run for all above functions
def data_conversion_from_pre_processed_data(pre_process_data,date_stg):
    # stg area
    final_data = []
    # flag
    flag_date_indx=0
    # cheker cols
    diff_appr_list = ['RPM','Fuel Rate (L-Hr)']
    
    # now apply data conversion on pre_processed_data
    for df in pre_process_data:
        print(f"\nData conversion for file --> {flag_date_indx} <--started...")
        print()
        # new dataframe creation
        new_df = pd.DataFrame(columns=name[:-1])
        # latitude values
        new_df['Latitude'] = df['Latitude'].apply(lat_convs)
        # longitude vals
        new_df['Longitude'] = df['Longitude'].apply(long_convs)
        # altitude vals
        new_df['Altitude'] = df['Altitude']
        # date vals
        new_df['Date'] = date_stg[flag_date_indx]
        
        # loop over all the values and for conversion
        for indx in range(len(all_col_names)):
            # stg of all the col name
            item = all_col_names[indx]
            
            # items other than base cols name
            if item not in diff_appr_list:
                # base item conversion
                new_df[item] = df[item].apply(ConvHexToDec, args=(multiplier_list[indx], offset[indx]))
                # cheking of cols <-and-> it's multipliers and offset
                #print(f"------> {item} <-multiplier_value-> {multiplier_list[indx]} <-offset_value-> {offset[indx]}")
                #print("*"*50)
                #print()
                
            # fuel cols conversion
            elif item=='Fuel Rate (L-Hr)':
                new_df[item] = df[item].apply(convSingleCol, args=(multiplier_list[indx], offset[indx]))
                # cheking of cols <-and-> it's multipliers and offset
                #print(f"------> {item} <-multiplier_value-> {multiplier_list[indx]} <-offset_value-> {offset[indx]}")
                #print("*"*50)
                #print()
                
            # other items conversion
            else:
                new_df[item] = df[item].apply(rpm_cal, args=(multiplier_list[indx], offset[indx]))
                #print(f"------> {item} <-multiplier_value-> {multiplier_list[indx]} <-offset_value-> {offset[indx]}")
        # param cols
        new_df['param1'] = df['param1']
        new_df['param2'] = df['param2']
        # final dataframe stg
        final_data.append(new_df)
        
        print(f"Data conversion completed for file --> {flag_date_indx}")
        flag_date_indx+=1
        print("#############################")
        print()
    return final_data

def saving_data_to_excel(data):
    file_names = ['T_000355.xlsx','T_000356.xlsx', 'T_000357.xlsx', 'T_000358.xlsx', 'T_000359.xlsx', 
                 'T_000360.xlsx', 'T_000361.xlsx', 'T_000362.xlsx', 'T_000363.xlsx', 'T_000364.xlsx', 
                 'T_000365.xlsx', 'T_000366.xlsx', 'T_000367.xlsx', 'T_000368.xlsx', 'T_000369.xlsx', 
                 'T_000370.xlsx', 'T_000371.xlsx', 'T_000372.xlsx', 'T_000373.xlsx']

    for indx in range(len(data)):
        file_name = 'data_preprocessed/'+ str(file_names[indx])
        data[indx].to_excel(file_name,index=False)
        print(f'Data of the file {file_name} saved successfully')
        print()
    return 0
    

# pipeline the data preprocessing steps
if __name__ == "__main__":
    # mention the directory from where to read all csv file
    directoryPath = ''
    # storage of all csv files using glob
    read_data_stg = []
    
    # read_csv_file_data from the above mentioned directory path
    read_data = data_reader()
    
    # pre-processed_data 
    pre_process_data,date_stg = pre_processing_data_and_date_finder(read_data)
    
    # and perform conversion operations
    all_data_file = data_conversion_from_pre_processed_data(pre_process_data,date_stg)
    
    # final data saving to excel
    saving_data_to_excel(all_data_file)


Reading Complete --->> T_000355.CSV
**********

Reading Complete --->> T_000356.CSV
**********

Reading Complete --->> T_000357.CSV
**********

Reading Complete --->> T_000358.CSV
**********

Reading Complete --->> T_000359.CSV
**********

Reading Complete --->> T_000360.CSV
**********

Reading Complete --->> T_000361.CSV
**********

Reading Complete --->> T_000362.CSV
**********

Reading Complete --->> T_000363.CSV
**********

Reading Complete --->> T_000364.CSV
**********

Reading Complete --->> T_000365.CSV
**********

Reading Complete --->> T_000366.CSV
**********

Reading Complete --->> T_000367.CSV
**********

Reading Complete --->> T_000368.CSV
**********

Reading Complete --->> T_000369.CSV
**********

Reading Complete --->> T_000370.CSV
**********

Reading Complete --->> T_000371.CSV
**********

Reading Complete --->> T_000372.CSV
**********

Reading Complete --->> T_000373.CSV
**********

Pre-processing file -->  0
Pre-processing file -->  1
Pre-processing file -->  2
Pre-pro